<a href="https://colab.research.google.com/github/hun9008/AI_TeamProject_24FW-/blob/main/Swin_trasformer_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.1 MB/s eta 0:00:00


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from timm import create_model

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Swin Transformer는 224x224 입력
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:18<00:00, 9101830.78it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
swin_transforemr = create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [8]:
print(swin_transforemr)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
            (drop1): 

In [9]:
window_attention = swin_transforemr.layers[0].blocks[0].attn

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
from torch import nn
class CustomWindowAttention(nn.Module):
    def __init__(self, original_attention, output_dim):
        super().__init__()
        self.qkv = original_attention.qkv
        self.softmax = original_attention.softmax
        self.cosine_similarity = nn.CosineSimilarity(dim=-1)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(output_dim, output_dim)
    def forward(self, x, mask=None):
        B, N, C = x.shape
        # qkv 계산 및 q, k, v로 분할
        qkv = self.qkv(x).reshape(B, N, 3, C).permute(2, 0, 1, 3)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # 코사인 유사도를 계산 (q와 k는 마지막 차원에서 코사인 유사도 계산)
        attn = nn.CosineSimilarity(dim=-1)(q.unsqueeze(2), k.unsqueeze(1)) / (C ** 0.5)
        attn = self.softmax(attn)

        output = torch.matmul(attn, v)
        output = self.linear(output)
        output = self.dropout(output)

        output = output + x
        return output

In [12]:
swin_transforemr.layers[0].blocks[0].attn = CustomWindowAttention(swin_transforemr.layers[0].blocks[0].attn, 96)
swin_transforemr.layers[0].blocks[1].attn = CustomWindowAttention(swin_transforemr.layers[0].blocks[1].attn, 96)
swin_transforemr.layers[1].blocks[0].attn = CustomWindowAttention(swin_transforemr.layers[1].blocks[0].attn, 192)
swin_transforemr.layers[1].blocks[1].attn = CustomWindowAttention(swin_transforemr.layers[1].blocks[1].attn, 192)
swin_transforemr.layers[2].blocks[0].attn = CustomWindowAttention(swin_transforemr.layers[2].blocks[0].attn, 384)
swin_transforemr.layers[2].blocks[1].attn = CustomWindowAttention(swin_transforemr.layers[2].blocks[1].attn, 384)
swin_transforemr.layers[2].blocks[2].attn = CustomWindowAttention(swin_transforemr.layers[2].blocks[2].attn, 384)
swin_transforemr.layers[2].blocks[3].attn = CustomWindowAttention(swin_transforemr.layers[2].blocks[3].attn, 384)
swin_transforemr.layers[2].blocks[4].attn = CustomWindowAttention(swin_transforemr.layers[2].blocks[4].attn, 384)
swin_transforemr.layers[2].blocks[5].attn = CustomWindowAttention(swin_transforemr.layers[2].blocks[5].attn, 384)
swin_transforemr.layers[3].blocks[0].attn = CustomWindowAttention(swin_transforemr.layers[3].blocks[0].attn, 768)
swin_transforemr.layers[3].blocks[1].attn = CustomWindowAttention(swin_transforemr.layers[3].blocks[1].attn, 768)

In [13]:
model = swin_transforemr.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [14]:
for epoch in range(10):
    model.train()
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predicted = torch.max(outputs.to(device), 1)[1]
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            correct += (labels == predicted).sum()
    print(
        f"epoch {epoch+1} - test loss: {test_loss / len(testloader):.4f}, accuracy: {correct / len(testloader):.4f}"
    )

KeyboardInterrupt: 